<a href="https://colab.research.google.com/github/MariMurotani/ColabNotebooks/blob/main/recipe_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Sat Oct 19 09:39:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                            

In [ ]:
# 必要なライブラリをインポートします。
!pip install transformers datasets
!pip install peft
!pip install trl
!pip install huggingface_hub # If you haven't installed huggingface_hub yet
!pip install google.colab
!pip install wandb -qU
!pip install bitsandbytes
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu124

#!pip install bitsandbytes
from google.colab import userdata
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments, Trainer, LlamaTokenizer, AutoTokenizer
import json
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from huggingface_hub import notebook_login
from google.colab import userdata
from huggingface_hub import login
import os, platform
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

from huggingface_hub import login
import wandb
import random
import math

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.8 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org

In [ ]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())

print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: NVIDIA A100-SXM4-40GB
Compute Capability: (8, 0)
Total Memory: 42481811456 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 6.1.85+
Python Version: 3.10.12


In [ ]:
# Secretにアクセス
access_token = userdata.get('HUGGING_FACE_TOKEN')
login(token=access_token)

# Wan dbログイン
access_token2 = userdata.get('WANDB_TOKEN')
wandb.login(key=access_token2)

# 保存モデルパス
base_path = "drive/MyDrive/RecipeTraining/LLM/"
source_path = f"{base_path}Source/"
model_path = f"{base_path}LLM/Output/my_fine_tuned_model"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# モデルとトークナイザーをロードします。
model_name = "meta-llama/Llama-3.2-3B-Instruct"

quantizated_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# pipelineを作成
pipe = pipeline(
    "text-generation",
    model=quantizated_model,
    tokenizer=tokenizer,
    device=0
)

In [ ]:
# Example prompt
prompt = "What can I cook with beef?"

# ハルシネーションの度合いをみてtempretureを調整する、低いほど正確
# Generate text
result = pipe(
    prompt, max_length=300, num_return_sequences=3, do_sample=True,
    temperature=0.7, top_p=0.9)
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'What can I cook with beef? and the most popular beef dishes from around the world?\nBeef is a versatile and delicious protein that can be used in a wide variety of dishes, from hearty stews and stir-fries to flavorful curries and skewers. Here are some popular beef dishes from around the world:\n\n**Global Beef Dishes**\n\n1.  **Korean Bibimbap**: A rice bowl dish topped with an assortment of vegetables, meat (usually beef), and a fried egg.\n2.  **Japanese Teriyaki Beef**: A sweet and savory dish featuring thinly sliced beef cooked in a sweet teriyaki sauce and served with steamed rice.\n3.  **Indian Beef Vindaloo**: A spicy curry made with marinated beef, vinegar, and a blend of spices.\n4.  **Mexican Carne Asada**: Thinly sliced grilled beef, often served with beans, rice, and tortillas.\n5.  **Chinese Beef and Broccoli Stir-Fry**: A classic stir-fry made with beef, broccoli, and a savory sauce, served over rice.\n6.  **Thai Massaman Beef Curry**: A rich and fla

## ↓↓↓↓ ここからFineTuningを実施する ↓↓↓↓

In [ ]:
# レシピデータの読み取り
source_path = "/content/drive/MyDrive/RecipeTraining/LLM/Source/"

def load_data(file_names):
  file_names = [f"{source_path}{file_name}" for file_name in file_names]
  train_texts = []
  for file_name in file_names:
    # データセットをロードします。
    with open(file_name, 'r') as f:
        recipes = json.load(f)
        # レシピのテキストを抽出します。
        for recipe in recipes:
          if recipe['title'] == "":
            continue

          prompt = recipe['prompt'] + "\n"
          text = "Title: " +  recipe['title'] + "\n"
          text += "Ingredients: \n"
          for ingredient in recipe['ingredients']:
              text += ingredient + "\n"
          text += "\n"
          text += "Instructions: \n"

          if recipe['instructions'][0] is not None:
            for instruction in recipe['instructions']:
                text += instruction + "\n"

          train_texts.append({"instruction": prompt, "input": "", "output": text , "text": prompt + "\n" + text})

  return train_texts

files = [
"Beef_Recipes.json", "German_Recipes.json", "Mexican_Recipes.json", "Seafood_Recipes.json",
"Chicken_Recipes.json", "Greek_Recipes.json", "Pasta_and_Noodle_Recipes.json", "Vegetable_Recipes.json",
"Fruit_Recipes.json", "Italian_Recipes.json", "Pork_Recipes.json"
]
train_texts = load_data(files)

# Input data into data set
dataset = Dataset.from_list(train_texts)
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 678
})


In [ ]:
# ↓↓↓↓↓↓↓　トレーニング実施 　 ↓↓↓↓↓↓↓
# ↓↓↓↓↓↓↓　↓↓↓↓↓↓↓ ↓↓↓↓↓↓↓↓
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

# Load the model and tokenizer with quantization
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Setup pipeline with GPU support
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

os.environ["WANDB_MODE"] = "offline"
wandb.login(key=access_token2)
wandb.init(project="my-finetuning-project", name="llama-finetuning")
peft_config = LoraConfig(
    lora_alpha= 8,
    lora_dropout= 0.1,
    r= 16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 1,
    per_device_train_batch_size= 8,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 1000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "linear",
    report_to="wandb",
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)
trainer.train()
trainer.model.save_pretrained(model_path)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

wandb: Currently logged in as: muromari5123 (muromari5123-none). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/678 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[at

Step,Training Loss
30,1.603200


In [ ]:
source_path = "/content/drive/MyDrive/RecipeTraining/LLM/OutPut/"
tokenizer.save_pretrained(model_path)
trainer.model.save_pretrained(source_path)

# 量子化
torch.quantization.convert(model, inplace=True)

# 量子化されたモデルの保存
torch.save(model.state_dict(), f"{source_path}static_quantized_model.pth")

In [ ]:
# Example prompt
prompt = "What can I cook with beef and cheese?"

# Generate text
result = pipe(
    prompt, max_length=300, num_return_sequences=3, do_sample=True,
    temperature=0.7, top_p=0.9)
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **

[{'generated_text': 'What can I cook with beef and cheese? \nTitle: Spinach and Feta Stuffed Chicken Breasts\nIngredients: \n1.5 pounds boneless, skinless chicken breasts\n1/2 cup fresh spinach, chopped\n1/4 cup feta cheese, crumbled\n1/2 cup shredded mozzarella\n1/4 cup grated Parmesan\n1 teaspoon dried oregano\n1 teaspoon dried thyme\n1/4 teaspoon ground black pepper\n1/4 teaspoon garlic powder\n1 tablespoon olive oil\n1/4 teaspoon salt\n1/2 cup panko breadcrumbs\n\nInstructions: \nPreheat oven to 375 degrees F (190 degrees C).\nRinse chicken breasts and pat dry with paper towels.\nIn a large bowl, combine spinach, feta cheese, mozzarella, Parmesan, oregano, thyme, black pepper, garlic powder, and olive oil. Mix well.\nGently stuff each chicken breast with the spinach and feta mixture.\nSeason chicken with salt. Place chicken on a baking sheet lined with parchment paper.\nIn a small bowl, mix together panko breadcrumbs and a pinch of salt.\nSprinkle both sides of the chicken with bre